In [1]:
import time
import torch
from torch.distributions import Categorical, kl
from d2l.torch import Animator

from net import Net
from aco import ACO
from utils import gen_pyg_data, load_test_dataset

torch.manual_seed(12345)
WD, WT = 0.7, 0.3
EPS = 1e-10
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [2]:
print(device)

cuda:0


In [3]:
@torch.no_grad()
def infer_instance(model, pyg_data, distances, time_matrix, n_ants,
                   t_aco_diff, k_sparse=None):
    if model:
        model.eval()
        heu_vec = model(pyg_data)
        heu_mat = model.reshape(pyg_data, heu_vec) + EPS
        aco = ACO(
            distances   = distances,
            travel_time = time_matrix,
            w_d         = WD,
            w_t         = WT,
            n_ants      = n_ants,
            heuristic   = heu_mat,
            device      = device
        )
    else:
        aco = ACO(
            distances   = distances,
            travel_time = time_matrix,
            w_d         = WD,
            w_t         = WT,
            n_ants      = n_ants,
            device      = device
        )
        if k_sparse:
            aco.sparsify(k_sparse)

    results = torch.zeros(len(t_aco_diff), device=device)
    for i, t in enumerate(t_aco_diff):
        results[i] = aco.run(t)
    return results
        
    
@torch.no_grad()
def test(dataset, model, n_ants, t_aco, k_sparse=None):
    _t_aco   = [0] + t_aco
    t_diffs  = [_t_aco[i+1]-_t_aco[i] for i in range(len(_t_aco)-1)]
    sum_res  = torch.zeros(len(t_diffs), device=device)
    start    = time.time()
    for pyg_data, distances, time_matrix in dataset:
        sum_res += infer_instance(model, pyg_data,
                                   distances, time_matrix,
                                   n_ants, t_diffs, k_sparse)
    return sum_res / len(dataset), time.time()-start

### Test on TSP20

DeepACO

In [4]:
n_ants = 20
n_node = 20
k_sparse = 10
t_aco = [1, 10, 20, 30, 40, 50, 100]
test_list = load_test_dataset(n_node, k_sparse, device)
net_tsp = Net().to(device)
net_tsp.load_state_dict(torch.load(f'../pretrained/tsp/tsp{n_node}.pt', map_location=device))
avg_aco_best, duration = test(test_list, net_tsp, n_ants, t_aco, k_sparse)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))    

total duration:  3037.9496920108795
T=1, average cost is 3.365255355834961.
T=10, average cost is 3.2843754291534424.
T=20, average cost is 3.278249740600586.
T=30, average cost is 3.275728225708008.
T=40, average cost is 3.2743453979492188.
T=50, average cost is 3.273648977279663.
T=100, average cost is 3.2715003490448.


ACO

In [5]:
n_ants = 20
n_node = 20
k_sparse = 10
t_aco = [1, 10, 20, 30, 40, 50, 100]
test_list = load_test_dataset(n_node, k_sparse, device)
avg_aco_best, duration = test(test_list, None, n_ants, t_aco, k_sparse)
print('total duration: ', duration)
for i, t in enumerate(t_aco):
    print("T={}, average cost is {}.".format(t, avg_aco_best[i]))    

total duration:  2987.522205352783
T=1, average cost is 4.481498718261719.
T=10, average cost is 3.610553503036499.
T=20, average cost is 3.4587457180023193.
T=30, average cost is 3.392754077911377.
T=40, average cost is 3.3546273708343506.
T=50, average cost is 3.3335437774658203.
T=100, average cost is 3.2924916744232178.
